In [96]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(rstatix)
library(parallel)
library(ggpubr)
library(ComplexHeatmap)
library(compositions)
library(ggsci)
library(math)

ERROR: Error in library(compositions): there is no package called ‘compositions’


In [3]:
colors = c('#FDC086','#FFFF99','#F0027F','#BF5B17','#666666',
          '#1B9E77','#D95F02','#7570B3','#E7298A','#66A61E','#E6AB02','#A6761D','#666666',
          '#A6CEE3','#1F78B4','#B2DF8A','#33A02C','#FB9A99','#E31A1C','#FDBF6F','#FF7F00',
          '#CAB2D6','#6A3D9A','#FFFF99','#B15928','#FBB4AE','#B3CDE3','#CCEBC5','#DECBE4',
          '#FED9A6','#FFFFCC','#E5D8BD','#FDDAEC','#F2F2F2','#B3E2CD','#FDCDAC','#CBD5E8',
          '#F4CAE4','#E6F5C9','#FFF2AE','#F1E2CC','#CCCCCC','#E41A1C','#377EB8','#4DAF4A',
          '#984EA3','#FF7F00','#FFFF33','#A65628','#F781BF','#999999','#66C2A5','#FC8D62',
          '#8DA0CB','#E78AC3','#A6D854','#FFD92F','#E5C494','#B3B3B3','#8DD3C7','#FFFFB3',
          '#BEBADA','#FB8072','#80B1D3','#FDB462','#B3DE69','#FCCDE5','#D9D9D9','#BC80BD',
          '#CCEBC5','#FFED6F')

# Reading the file - cross sectional

In [4]:
# Read the CSV file into a data frame
myeloid_df <- read.csv('/home/jupyter/Myeloid_cells/files/Parquet_files/AIFI_Reference_processed.csv')


In [5]:
#remove UPI from the cohort
myeloid_df <- myeloid_df %>% filter(cohort.cohortGuid != 'UP1')

In [7]:
head(myeloid_df)
colnames(myeloid_df)

,barcodes,barcodes.1,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,⋯,pct_counts_mito,leiden,leiden_resolution_1,leiden_resolution_1.5,leiden_resolution_2,AIFI_L1,AIFI_L2,AIFI_L3,AIFI_L3.5,cluster_dummy
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,cf71f47048b611ea8957bafe6d70929e,cf71f47048b611ea8957bafe6d70929e,B001,weathered_pernicious_polliwog,cf71f47048b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1081,115,⋯,4.032258,10,9,11,10,T cell,MAIT,CD8 MAIT,CD8 MAIT,False
2,cf71fb7848b611ea8957bafe6d70929e,cf71fb7848b611ea8957bafe6d70929e,B001,long_weakminded_roebuck,cf71fb7848b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1118,77,⋯,2.071006,2,7,4,3,T cell,Memory CD4 T cell,CM CD4 T cell,CM CD4 T cell,False
3,cf7216a848b611ea8957bafe6d70929e,cf7216a848b611ea8957bafe6d70929e,B001,bimetallic_returnable_pony,cf7216a848b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,1836,249,⋯,4.098091,2,6,8,6,T cell,Memory CD4 T cell,GZMB- CD27+ EM CD4 T cell,GZMB- CD27+ EM CD4 T cell,False
4,cf72178448b611ea8957bafe6d70929e,cf72178448b611ea8957bafe6d70929e,B001,newsworthy_copacetic_halicore,cf72178448b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,928,60,⋯,2.130682,1,0,1,0,T cell,Naive CD4 T cell,Core naive CD4 T cell,Core naive CD4 T cell,False
5,cf721a4048b611ea8957bafe6d70929e,cf721a4048b611ea8957bafe6d70929e,B001,esthetical_diffident_mosasaur,cf721a4048b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,2832,219,⋯,2.766199,25,16,21,26,NK cell,Proliferating NK cell,Proliferating NK cell,Proliferating NK cell,False
6,cf7224c248b611ea8957bafe6d70929e,cf7224c248b611ea8957bafe6d70929e,B001,chic_mahogany_caribou,cf7224c248b611ea8957bafe6d70929e,B001-P1C1,TGATGGCCTATTGGG,singlet,951,122,⋯,5.640314,7,2,6,4,T cell,Naive CD8 T cell,Core naive CD8 T cell,Core naive CD8 T cell,False


[1] "barcodes"                              
 [2] "barcodes.1"                            
 [3] "batch_id"                              
 [4] "cell_name"                             
 [5] "cell_uuid"                             
 [6] "chip_id"                               
 [7] "hto_barcode"                           
 [8] "hto_category"                          
 [9] "n_genes"                               
[10] "n_mito_umis"                           
[11] "n_reads"                               
[12] "n_umis"                                
[13] "original_barcodes"                     
[14] "pbmc_sample_id"                        
[15] "pool_id"                               
[16] "seurat_pbmc_type"                      
[17] "seurat_pbmc_type_score"                
[18] "umap_1"                                
[19] "umap_2"                                
[20] "well_id"                               
[21] "subject.biologicalSex"                 
[22] "subject.ethnicity"                     
[23] "subject.partnerCode"                   
[24] "subject.race"                          
[25] "subject.subjectGuid"                   
[26] "cohort.cohortGuid"                     
[27] "sample.visitName"                      
[28] "sample.visitDetails"                   
[29] "subject.birthYear"                     
[30] "CMV.IgG.Serology.Result.Interpretation"
[31] "BMI"                                   
[32] "predicted.celltype.l1.score"           
[33] "predicted.celltype.l1"                 
[34] "predicted.celltype.l2.score"           
[35] "predicted.celltype.l2"                 
[36] "predicted.celltype.l3.score"           
[37] "predicted.celltype.l3"                 
[38] "predicted.celltype.l2.5.score"         
[39] "predicted.celltype.l2.5"               
[40] "predicted_labels_celltypist"           
[41] "majority_voting_celltypist"            
[42] "predicted_doublet"                     
[43] "doublet_score"                         
[44] "n_genes_by_counts"                     
[45] "log1p_n_genes_by_counts"               
[46] "total_counts"                          
[47] "log1p_total_counts"                    
[48] "pct_counts_in_top_50_genes"            
[49] "pct_counts_in_top_100_genes"           
[50] "pct_counts_in_top_200_genes"           
[51] "pct_counts_in_top_500_genes"           
[52] "total_counts_mito"                     
[53] "log1p_total_counts_mito"               
[54] "pct_counts_mito"                       
[55] "leiden"                                
[56] "leiden_resolution_1"                   
[57] "leiden_resolution_1.5"                 
[58] "leiden_resolution_2"                   
[59] "AIFI_L1"                               
[60] "AIFI_L2"                               
[61] "AIFI_L3"                               
[62] "AIFI_L3.5"                             
[63] "cluster_dummy"

In [11]:
unique(myeloid_df$pbmc_sample_id)

[1] "PB00001-01" "PB00002-01" "PB00003-01" "PB00004-01" "PB00006-01"
 [6] "PB00010-02" "PB00012-01" "PB00014-01" "PB00015-01" "PB00016-01"
[11] "PB00022-01" "PB00023-05" "PB00025-04" "PB00026-05" "PB00030-02"
[16] "PB00031-05" "PB00041-01" "PB00166-01" "PB00334-01" "PB00338-01"
[21] "PB00339-01" "PB00341-01" "PB00342-01" "PB00345-01" "PB00347-01"
[26] "PB00349-01" "PB00350-01" "PB00352-04" "PB00353-04" "PB00356-01"
[31] "PB00357-01" "PB00362-01" "PB00363-01" "PB00365-01" "PB00366-01"
[36] "PB00368-07" "PB00369-01" "PB00377-01" "PB00383-01" "PB00384-01"
[41] "PB00386-01" "PB00387-01" "PB00388-01" "PB00390-01" "PB00391-01"
[46] "PB00393-01" "PB00394-02" "PB00398-01" "PB00501-01" "PB00504-01"
[51] "PB00507-01" "PB00510-01" "PB00514-01" "PB00516-01" "PB00518-01"
[56] "PB00520-01" "PB00523-01" "PB00525-01" "PB00526-01" "PB00529-01"
[61] "PB00530-01" "PB00532-01" "PB00533-01" "PB00535-01" "PB00537-01"
[66] "PB00539-01" "PB00541-01" "PB00545-01" "PB00549-01" "PB00552-01"
[71] "PB00557-01" "PB00561-04" "PB00562-01" "PB00563-01" "PB00564-04"
[76] "PB00565-01" "PB00566-01" "PB00568-01" "PB00569-01" "PB00575-01"
[81] "PB00582-01" "PB00588-01" "PB00592-01" "PB00599-01" "PB00600-01"
[86] "PB00602-01" "PB00621-01" "PB00623-01" "PB00625-02" "PB00645-01"
[91] "PB01425-01" "PB01446-01"

In [8]:
rownames(myeloid_df)<-myeloid_df$cell_uuid

In [ ]:
#s elected: "subject.subjectGuid", "cohort.cohortGuid", "CMV.IgG.Serology.Result.Interpretation", "pbmc_sample_id", "subject.biologicalSex", "subject.birthYear", "subject.ethnicity", "subject.partnerCode", "subject.race", "

In [28]:
## Percentage of total

result <- myeloid_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid ) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) 
  

head(result)
#sum(result$subject.subjectGuid)

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


AIFI_L3,subject.subjectGuid,count,percentage
<chr>,<chr>,<int>,<dbl>
ASDC,BR1001,6,0.03236770
ASDC,BR1002,5,0.02637409
ASDC,BR1003,4,0.02094680
ASDC,BR1004,3,0.01710571
ASDC,BR1005,5,0.02809620
ASDC,BR1006,4,0.02300966


In [42]:
result %>% filter(subject.subjectGuid == "BR1024") %>% 
      select(percentage) %>%
      pull() %>% 
      sum()

Adding missing grouping variables: `subject.subjectGuid`


[1] 100

In [31]:

result_new<-left_join(result,unique(myeloid_df[c("subject.subjectGuid", "cohort.cohortGuid", "CMV.IgG.Serology.Result.Interpretation", "pbmc_sample_id", "subject.biologicalSex", "subject.birthYear", "subject.ethnicity", "subject.partnerCode", "subject.race", "sample.visitName")]),by=c('subject.subjectGuid'))



In [32]:
head(result_new)

AIFI_L3,subject.subjectGuid,count,percentage,cohort.cohortGuid,CMV.IgG.Serology.Result.Interpretation,pbmc_sample_id,subject.biologicalSex,subject.birthYear,subject.ethnicity,subject.partnerCode,subject.race,sample.visitName
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
ASDC,BR1001,6,0.03236770,BR1,Negative,PB00001-01,Female,1987,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0
ASDC,BR1002,5,0.02637409,BR1,Negative,PB00002-01,Male,1991,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0
ASDC,BR1003,4,0.02094680,BR1,Negative,PB00003-01,Female,1989,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0
ASDC,BR1004,3,0.01710571,BR1,Negative,PB00004-01,Male,1989,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0
ASDC,BR1005,5,0.02809620,BR1,Negative,PB00006-01,Female,1992,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0
ASDC,BR1006,4,0.02300966,BR1,Negative,PB00010-02,Male,1988,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0


In [34]:
library(tidyverse)

# Assuming your dataframe is named 'df'

# Pivot the dataframe from long to wide format
wide_df <- result_new %>%
  pivot_wider(
    id_cols = c(cohort.cohortGuid,subject.subjectGuid, CMV.IgG.Serology.Result.Interpretation, pbmc_sample_id, subject.biologicalSex, subject.birthYear, subject.ethnicity,subject.partnerCode, subject.race, sample.visitName),   # Columns to keep as is
    names_from = AIFI_L3,          # Column to pivot from
    values_from = percentage       # Column to pivot to values
  )



In [37]:
head(wide_df)

cohort.cohortGuid,subject.subjectGuid,CMV.IgG.Serology.Result.Interpretation,pbmc_sample_id,subject.biologicalSex,subject.birthYear,subject.ethnicity,subject.partnerCode,subject.race,sample.visitName,⋯,Platelet,Proliferating NK cell,Proliferating T cell,SOX4+ Vd1 gdT,SOX4+ naive CD4 T cell,SOX4+ naive CD8 T cell,Transitional B cell,Type 2 polarized memory B cell,cDC1,pDC
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BR1,BR1001,Negative,PB00001-01,Female,1987,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.2589416,0.2049954,0.35065005,0.05934078,0.7714301,0.2373631,0.42617468,0.10789232,0.05394616,0.5340670
BR1,BR1002,Negative,PB00002-01,Male,1991,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.2109927,0.1635194,0.16879418,0.03692373,0.9863910,0.2268172,0.08439709,0.02109927,0.03692373,0.7859479
BR1,BR1003,Negative,PB00003-01,Female,1989,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.1571010,0.1256808,0.15186426,0.26707164,2.8435274,0.7174277,0.31943863,0.12568077,0.03142019,0.4241726
BR1,BR1004,Negative,PB00004-01,Male,1989,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.5188733,0.2337781,0.13684571,0.02280762,0.1995667,0.1311438,0.15965332,0.10263428,0.05131714,0.4732581
BR1,BR1005,Negative,PB00006-01,Female,1992,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.2809620,0.1180040,0.13486177,0.05057316,0.7585974,0.2135311,0.25848505,0.11238481,0.01123848,0.5450663
BR1,BR1006,Negative,PB00010-02,Male,1988,Non-Hispanic origin,BR,Caucasian,Flu Year 1 Day 0,⋯,0.2531063,0.1553152,0.06902899,0.01150483,0.8513576,0.2473539,0.41992637,0.16682006,0.06902899,0.5752416


In [36]:
unique(wide_df$subject.subjectGuid)

[1] "BR1001" "BR1002" "BR1003" "BR1004" "BR1005" "BR1006" "BR1007" "BR1008"
 [9] "BR1009" "BR1011" "BR1012" "BR1013" "BR1014" "BR1015" "BR1016" "BR1017"
[17] "BR1018" "BR1019" "BR1021" "BR1023" "BR1024" "BR1025" "BR1026" "BR1030"
[25] "BR1031" "BR1032" "BR1033" "BR1037" "BR1040" "BR1041" "BR1042" "BR1043"
[33] "BR1044" "BR1045" "BR1047" "BR1048" "BR1049" "BR1050" "BR1051" "BR1052"
[41] "BR1054" "BR1056" "BR1057" "BR1058" "BR1059" "BR2002" "BR2003" "BR2005"
[49] "BR2008" "BR2009" "BR2010" "BR2012" "BR2013" "BR2014" "BR2015" "BR2016"
[57] "BR2019" "BR2021" "BR2022" "BR2023" "BR2024" "BR2025" "BR2026" "BR2028"
[65] "BR2029" "BR2030" "BR2031" "BR2032" "BR2033" "BR2034" "BR2035" "BR2036"
[73] "BR2037" "BR2038" "BR2039" "BR2041" "BR2042" "BR2043" "BR2044" "BR2045"
[81] "BR2046" "BR2047" "BR2048" "BR2050" "BR2052" "BR2053" "BR1028" "BR2018"
[89] "BR2020" "BR2051" "BR1010" "BR2040"

In [39]:
# Assuming your dataframe is named 'df' and you want to save it as 'output.csv'

write.csv(wide_df, file = "Frequency_Percentage_table_L3.csv", row.names = TRUE)


# Reading the file - Longitudinal

In [43]:
# Read the CSV file into a data frame (No UP1)
flu_df <- read.csv('/home/jupyter/Myeloid_cells/files/AIFI_fluYr1Yr2_raw.csv')


In [48]:
colnames(flu_df)

[1] "barcodes"                              
 [2] "barcodes.1"                            
 [3] "batch_id"                              
 [4] "cell_name"                             
 [5] "cell_uuid"                             
 [6] "chip_id"                               
 [7] "hto_barcode"                           
 [8] "hto_category"                          
 [9] "n_genes"                               
[10] "n_mito_umis"                           
[11] "n_reads"                               
[12] "n_umis"                                
[13] "original_barcodes"                     
[14] "pbmc_sample_id"                        
[15] "pool_id"                               
[16] "seurat_pbmc_type"                      
[17] "seurat_pbmc_type_score"                
[18] "umap_1"                                
[19] "umap_2"                                
[20] "well_id"                               
[21] "subject.biologicalSex"                 
[22] "subject.ethnicity"                     
[23] "subject.partnerCode"                   
[24] "subject.race"                          
[25] "subject.subjectGuid"                   
[26] "cohort.cohortGuid"                     
[27] "sample.visitName"                      
[28] "sample.visitDetails"                   
[29] "subject.birthYear"                     
[30] "CMV.IgG.Serology.Result.Interpretation"
[31] "BMI"                                   
[32] "predicted_doublet"                     
[33] "doublet_score"                         
[34] "AIFI_L1"                               
[35] "AIFI_L2"                               
[36] "AIFI_L3"                               
[37] "AIFI_L3.5"                             
[38] "n_genes_by_counts"                     
[39] "log1p_n_genes_by_counts"               
[40] "total_counts"                          
[41] "log1p_total_counts"                    
[42] "pct_counts_in_top_50_genes"            
[43] "pct_counts_in_top_100_genes"           
[44] "pct_counts_in_top_200_genes"           
[45] "pct_counts_in_top_500_genes"           
[46] "total_counts_mito"                     
[47] "log1p_total_counts_mito"               
[48] "pct_counts_mito"

In [44]:
rownames(flu_df)<-flu_df$cell_uuid

In [83]:
## Percentage of total

result <- flu_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid,sample.visitName ) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid, sample.visitName) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) 
  

head(result)
#sum(result$subject.subjectGuid)

`summarise()` has grouped output by 'AIFI_L3', 'subject.subjectGuid'. You can
override using the `.groups` argument.


AIFI_L3,subject.subjectGuid,sample.visitName,count,percentage
<chr>,<chr>,<chr>,<int>,<dbl>
ASDC,BR1001,Flu Year 1 Day 0,6,0.03280481
ASDC,BR1001,Flu Year 1 Day 7,5,0.02530108
ASDC,BR1002,Flu Year 1 Day 0,3,0.01695490
ASDC,BR1002,Flu Year 1 Day 7,2,0.01150616
ASDC,BR1002,Flu Year 2 Day 0,3,0.02000400
ASDC,BR1002,Flu Year 2 Day 7,2,0.01236247


In [85]:
result %>% filter(subject.subjectGuid == "BR1001") %>% 
      select(percentage) %>%
      pull() %>% 
      sum()

Adding missing grouping variables: `subject.subjectGuid`, `sample.visitName`


[1] 200

In [86]:
length(result$AIFI_L3)

[1] 23990

In [88]:

result_new<-inner_join(result,unique(flu_df[c("subject.subjectGuid", "cohort.cohortGuid", "CMV.IgG.Serology.Result.Interpretation", "pbmc_sample_id", "subject.biologicalSex", "subject.birthYear", "subject.ethnicity", "subject.partnerCode", "subject.race", "sample.visitName", "AIFI_L3")]),by=c('subject.subjectGuid',"sample.visitName", "AIFI_L3"))



In [89]:
length(result_new$AIFI_L3)

[1] 23990

In [90]:
library(tidyverse)

# Assuming your dataframe is named 'df'

# Pivot the dataframe from long to wide format
wide_df <- result_new %>%
  pivot_wider(
    id_cols = c(cohort.cohortGuid,subject.subjectGuid, sample.visitName, CMV.IgG.Serology.Result.Interpretation, pbmc_sample_id, subject.biologicalSex, subject.birthYear, subject.ethnicity,subject.partnerCode, subject.race),   # Columns to keep as is
    names_from = AIFI_L3,          # Column to pivot from
    values_from = percentage       # Column to pivot to values
  )



In [94]:
head(wide_df)

cohort.cohortGuid,subject.subjectGuid,sample.visitName,CMV.IgG.Serology.Result.Interpretation,pbmc_sample_id,subject.biologicalSex,subject.birthYear,subject.ethnicity,subject.partnerCode,subject.race,⋯,Platelet,Proliferating NK cell,Proliferating T cell,SOX4+ Vd1 gdT,SOX4+ naive CD4 T cell,SOX4+ naive CD8 T cell,Transitional B cell,Type 2 polarized memory B cell,cDC1,pDC
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BR1,BR1001,Flu Year 1 Day 0,Negative,PB00001-01,Female,1987,Non-Hispanic origin,BR,Caucasian,⋯,0.26243849,0.2186987,0.3608529,0.07107709,0.6834336,0.1968289,0.4373975,0.08201203,0.03827228,0.4975396
BR1,BR1001,Flu Year 1 Day 7,Negative,PB00008-01,Female,1987,Non-Hispanic origin,BR,Caucasian,⋯,0.28843234,0.2175893,0.3339743,0.05566238,0.8197551,0.1872280,0.6224066,0.06578282,0.04554195,0.3036130
BR1,BR1002,Flu Year 1 Day 0,Negative,PB00002-01,Male,1991,Non-Hispanic origin,BR,Caucasian,⋯,0.22606533,0.1695490,0.1695490,0.04521307,0.7742738,0.1752006,0.1130327,0.02260653,0.03390980,0.7968803
BR1,BR1002,Flu Year 1 Day 7,Negative,PB00009-01,Male,1991,Non-Hispanic origin,BR,Caucasian,⋯,0.11506156,0.1323208,0.1725923,0.04602462,0.7594063,0.2071108,0.2071108,0.02301231,0.02301231,0.9147394
BR1,BR1002,Flu Year 2 Day 0,Negative,PB00378-01,Male,1991,Non-Hispanic origin,BR,Caucasian,⋯,0.05334400,0.1066880,0.1333600,0.10002000,1.1002200,0.2800560,0.8601720,0.05334400,0.03334000,0.4934320
BR1,BR1002,Flu Year 2 Day 7,Negative,PB00400-01,Male,1991,Non-Hispanic origin,BR,Caucasian,⋯,0.04944987,0.1298059,0.1607121,0.07417481,0.9889974,0.2225244,0.8530103,0.04326864,0.03090617,0.5192236


In [92]:
unique(wide_df$subject.subjectGuid)

[1] "BR1001" "BR1002" "BR1003" "BR1004" "BR1005" "BR1006" "BR1007" "BR1008"
 [9] "BR1009" "BR1010" "BR1011" "BR1012" "BR1013" "BR1014" "BR1015" "BR1016"
[17] "BR1017" "BR1018" "BR1019" "BR1021" "BR1023" "BR1024" "BR1025" "BR1026"
[25] "BR1028" "BR1030" "BR1031" "BR1032" "BR1033" "BR1037" "BR1040" "BR1041"
[33] "BR1042" "BR1043" "BR1044" "BR1045" "BR1047" "BR1048" "BR1049" "BR1050"
[41] "BR1051" "BR1052" "BR1054" "BR1056" "BR1057" "BR1058" "BR1059" "BR2002"
[49] "BR2003" "BR2005" "BR2008" "BR2009" "BR2010" "BR2012" "BR2013" "BR2014"
[57] "BR2015" "BR2016" "BR2018" "BR2019" "BR2020" "BR2021" "BR2022" "BR2023"
[65] "BR2024" "BR2025" "BR2026" "BR2028" "BR2029" "BR2030" "BR2031" "BR2032"
[73] "BR2033" "BR2034" "BR2035" "BR2036" "BR2037" "BR2038" "BR2039" "BR2040"
[81] "BR2041" "BR2042" "BR2043" "BR2044" "BR2045" "BR2046" "BR2047" "BR2048"
[89] "BR2050" "BR2051" "BR2052" "BR2053"

In [95]:
# Assuming your dataframe is named 'df' and you want to save it as 'output.csv'

write.csv(wide_df, file = "Frequency_Percentage_table_L3_Flu_Yr1_Yr2_timepoints.csv", row.names = TRUE)
